In [5]:
import seaborn as sns
import matplotlib 

import matplotlib.pyplot as plt


import math
from sys import path
path.append('..//modules')
path.append('..//dumps//reg_dmo_pdf')
import time
import pandas as pd
import psycopg2 as pg
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import create_engine, MetaData, Table
import pandas.io.sql as psql

#create a postgres engine and a pg connection
from modules.db_connector import *
print("DB connecting engine imported")


#print(matplotlib.rcParams.keys())

font = {'family' : 'Roboto', 
        'weight': '300',
        'size'   : 20}


axesfoo = {'labelweight': '300'}
#unified_colors
clr_mean = "red"
clr_mode = "black"
clr_median = "orange"

import matplotlib 
matplotlib.rc('font', **font)
matplotlib.rc('axes', **axesfoo)

#https://medium.com/dunder-data/automatically-wrap-graph-labels-in-matplotlib-and-seaborn-a48740bc9ce
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                      break_long_words=break_long_words))
    ax.set_xticklabels(labels, rotation=0)


DB connecting engine imported


In [6]:
with engine.connect() as con1:
    q1 = 'select * from gtlab.sk_sgs_vgl_sguds'
    q2 = 'select * from gtlab.sk_pois_activities_tripadvisor_0723_details'
    df_sgs = pd.read_sql(q1, con=con1)
    df_ta_poi_all = pd.read_sql(q2, con=con1)

In [7]:
#https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

#vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')
vectorizer = TfidfVectorizer(tokenizer=normalize)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]
import string
from unidecode import unidecode


In [8]:
global l_df_simxy 
l_df_simxy = []

def get_name_penetration(df1, df2):
    for ix, vgl_id, nm_vgl, in zip(df1.index, df1['id'], df1['lokalita']):
        print(ix)
        for iy, poi_id, poi_name in zip(df2.index, df2['id'],df2['poi_name']):
            if unidecode(nm_vgl) == unidecode(poi_name):
                sim_xy = cosine_sim(unidecode(nm_vgl), unidecode(poi_name))
                print("similarity: ", sim_xy , ix, vgl_id, nm_vgl, iy, poi_name)
                l_df_simxy.append(pd.DataFrame([vgl_id, nm_vgl, poi_id, poi_name, sim_xy]).T)
            elif unidecode(nm_vgl) != unidecode(poi_name) and any(pnct in nm_vgl for pnct in string.punctuation):
                for ele in nm_vgl:
                    if ele in string.punctuation:
                        for part in nm_vgl.split(ele):
                            if cosine_sim(unidecode(part), unidecode(poi_name)) > 0.50:
                                sim_xy = cosine_sim(unidecode(part), unidecode(poi_name))
                                print("similarity: ", sim_xy , ix, vgl_id, nm_vgl, iy, poi_name)
                                l_df_simxy.append(pd.DataFrame([vgl_id, nm_vgl, poi_id, poi_name, sim_xy]).T)
            elif unidecode(nm_vgl) != poi_name and all(pnct not in nm_vgl for pnct in string.punctuation):
                if cosine_sim(unidecode(nm_vgl), unidecode(poi_name)) > 0.50:
                    sim_xy = cosine_sim(unidecode(nm_vgl), unidecode(poi_name))
                    print("similarity: ", sim_xy , ix, vgl_id, nm_vgl, iy, poi_name)
                    l_df_simxy.append(pd.DataFrame([vgl_id, nm_vgl, poi_id, poi_name, sim_xy]).T)
                    
                
                
get_name_penetration(df_sgs, df_ta_poi_all)            


0
1
2
similarity:  0.5797386715376657 2 78.0 Devín, hradná skala 2622 Devin Castle


C:\Users\csb\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


3
4
5
6
7
similarity:  0.5797386715376657 7 404.0 Dolná Súča, Krasín 903 Paintball klub Dolná Súča


C:\Users\csb\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


8
9
10
similarity:  0.5797386715376657 10 11.0 Kremnica 38 Mincovna Kremnica


...

466
467
468
469
470
similarity:  0.7092972666062738 470 116.0 Štrbské pleso 1885 Strbske Pleso Lake


C:\Users\csb\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


similarity:  0.9999999999999998 470 116.0 Štrbské pleso 2582 Strbske Pleso


C:\Users\csb\anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


471
472
473
474
475
476
477
478
479


In [26]:
df_sgs_tripadvisor = pd.concat(l_df_simxy)
df_sgs_tripadvisor['sgs_id'] = df_sgs_tripadvisor[0]
df_sgs_tripadvisor['sgs_name'] = df_sgs_tripadvisor[1]
df_sgs_tripadvisor['poi_id'] = df_sgs_tripadvisor[2]
df_sgs_tripadvisor['poi_name'] = df_sgs_tripadvisor[3]
df_sgs_tripadvisor['text_sim'] = df_sgs_tripadvisor[4]
df_sgs_tripadvisor = df_sgs_tripadvisor.drop([0,1,2,3,4], axis=1)


In [29]:
df_sgs_tripadvisor['ok'] = ['ok' if x >= 0.9 else None for x in df_sgs_tripadvisor['text_sim']]

In [38]:
sgs_ta_match = ['g274924-d290673', 'g799614-d8549004', 'g609045-d11512717', 'g9698122-d11664638', 'g17793409-d17791340', 'g14145204-d10837151',
'g23301823-d23300604', 'g1137995-d12856832', 'g8649014-d17380245', 'g799614-d8549004', 'g12182959-d24042843', 'g23301836-d23300667',
'g18721430-d23300680', 'g799612-d12837873', 'g799612-d12837873', 'g23301829-d23300650', 'g3175023-d2719150', 'g5288165-d10158815',
'g7718445-d25060057', 'g799594-d8544722', 'g285716-d12374708', 'g799606-d12442142', 'g12500843-d23300690', 'g7602660-d10898537',
'g982822-d8549787', 'g285716-d7698046']

df_sgs_tripadvisor = df_sgs_tripadvisor.query('poi_id in @sgs_ta_match')
df_sgs_tripadvisor.reset_index()

In [45]:
df_sgs_tripadvisor.to_sql('sgs_geosites_ta_penetration',  engine, schema='gtlab', if_exists='replace')

26